In [ ]:
# 모델과 문장 분리를 위한 토크나이저를 불러온다.
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("EbanLee/kobart-summary-v3")
model = AutoModelForSeq2SeqLM.from_pretrained("EbanLee/kobart-summary-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
kakao = pd.read_csv('/data/kakao_final.csv')
naver = pd.read_csv('/data/naver_final.csv')
print(len(kakao), len(naver))

175 697


In [ ]:
df = pd.concat([kakao, naver])
df['restaurant'].unique()

array(['무명칼국수', '연어시장 석계본점', '한상', '윤스쿡', '라체나1907'], dtype=object)

In [ ]:
restaurant_df = df[df['restaurant'] == '한상'] # 음식점 필터링
category_df = restaurant_df[restaurant_df['category'] == '가격']
category_df = category_df['review']

data = category_df[category_df.apply(lambda x: len(x.split()) > 2)].reset_index(drop=True) # 2단어 이하인 경우 삭제

text = " ".join(data.tolist()) # 리뷰를 하나의 문장으로 합치기
text = re.sub(r'[^가-힣\s]', '', text) # 한글, 숫자만 남김
text = " ".join(text.split()) # 불필요한 공백 제거

text

'엄마가놀래서 가방을 붙잡음 천만원짜리 가방임 가방가격이 중요한게 여기망할듯 음식금액을 올리고 마음공부하세요 가격은 비싼데 나오는 반찬들이 가지수만 많고 반찬 구성에 대해 말이 많던데 이 가격에 이 정도면 가성비 좋은 뻥 뚫린 식당같은 분위기와 이 가격에 종이컵은 개선 필요합니다 가격 대비 최악 별점 도 아깝다 가격대비 음식 구성 맛 너무 떨어진다 가격 대비 넘 좋았어요 불친절하다는 그냥 딱 이정도 가격의 느낌 지난번보다 가격은 올랐는데 반찬은불고기 제외됨 음식은 가격대비 조금 부족해 보이네요 가격도 요즘 시세에 비하면 적당했구요 솔직히 가격대비 별로입니다 가격만큼 반찬이나 메인메뉴가 부족한 듯 해요 가격대비 그냥그렇고 사장님 같은 같은 가격인데 평일하고 어떻게 음식 가격 높음 그 가격에 비해 좀 가격대비 약한듯하고 음식은 맛나 더 비싸 상차림과 술안주메뉴들도 더 다양해요 저렴한 가격 때문이라고 하기에는 음식수준에 비해 떡갈비도 수제가 아니구 가격대비 음식 구성이 허술해요 그냥 앉았는데 그 가격에 이런음식은 좀 아닌듯 하네요 다만 가격대비 엄청 맛있다거나 특색있는건 같네요 가격을 내리시던지요 가격대에 비해서 인지 가격대비 음식이 알차게 나오진 않은듯 그가격의 세트구성이라면 후식이 너무 약하달까 메실 가격대비 어중간한 강남권 식당들보다 퀄리티 있고 참고로 가격대비 여기보다 맛없고 퀄리티 떨어지는 곳 수두룩함 가격 대비 너무뻔한맛 가격만큼 음식이 고급은 아니네요 흔한 가격대비 만족도 굿입니다 가격 대비 음식이 아주 훌륭하지 않아서 또 멀리서 찾아가지는 가격을 만원대로 낮추시고 반찬수를 반으로 줄여서 맛나게'

In [ ]:
result = []

for restaurant in df['restaurant'].unique():
  restaurant_df = df[df['restaurant'] == restaurant] # 음식점 필터링

  for category in restaurant_df['category'].unique():
    category_df = restaurant_df[restaurant_df['category'] == category] # 카테고리 필터링
    category_df = category_df['review']

    data = category_df[category_df.apply(lambda x: len(x.split()) > 2)].reset_index(drop=True) # 2단어 이하인 경우 삭제

    text = " ".join(data.tolist()) # 리뷰를 하나의 문장으로 합치기
    text = re.sub(r'[^가-힣\s]', '', text) # 한글, 숫자만 남김
    text = " ".join(text.split()) # 불필요한 공백 제거

    input_ids = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)
    input_ids = torch.tensor([tokenizer.bos_token_id] + input_ids[0].tolist() + [tokenizer.eos_token_id]).unsqueeze(0)

    summary_text_ids = model.generate(
      input_ids=input_ids,
      bos_token_id=model.config.bos_token_id,
      eos_token_id=model.config.eos_token_id,
      length_penalty=1,  # 길이에 대한 penalty 값. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
      max_new_tokens=50,  # 요약문의 최대 길이 설정
      min_length=20,  # 요약문의 최소 길이 설정
      num_beams=8,  # 문장 생성 시 다음 단어를 탐색하는 영역의 개수
      early_stopping=True,  # 요약문이 더 짧은 경우 일찍 멈추기
      repetition_penalty=2.0  # 반복 패널티 추가
    )

    summary = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    cleaned_text = "\n".join([line for line in summary.splitlines() if line.strip()])

    result.append({'restaurant': restaurant, 'category': category, 'summary': cleaned_text, 'real': text})
    print(f'{restaurant}의 {category}카테고리 결과입니다.\n')
    print(f'[원문]\n{text}\n')
    print(f'[요약문]\n{cleaned_text}')
    print("==============================================================")


무명칼국수의 맛카테고리 결과입니다.
[원문]
맛 전혀 특별하지 않음 해물칼국수 정말 괜찮음 닭칼은 그닥 만두는 부추맛이 강해서 쫄깃한 식감의 수타면 아삭하고 감칠맛 나는 겉절이 반찬으로만 사가고 싶음 찐맛집 닭칼이랑 해물칼국수 둘다 면이랑 국물 미쳤어요 김치도 대존맛 만두도 진짜임 광운대 가면 무조건 가야되는 맛집 다 친절하시고 김치 맛 변했음 안 가 짜다 기본간이 짠데 꽤 맛좋은 집 해물칼국수 추천 우선 칼국수면발이 아주 쫄깃해서 식감이 좋다 석계 맛집 해물도 닭고 축복임 맛 서비스 모두 최고 해물 칼국수좋아요 만두도맛나요 석계역 가면 자주들리는집 국물이좋아서 솔직히 말하면 칼국수는 그냥 그저그런 맛인데 김치가 한몫함 칼국수면 직접 만드시는거 같은데 진짜 쫄깃하고 국물도 시원함 겉절이가 신의 한수 번 리필함 만두는 조미료맛 없이 집에서 만든 담백한 맛이라 호불호 갈릴듯 손칼국수라는거 말고는 맛의 특별함이 없는 집 만두맛도 평범하고 같이 간 지인에게 또 방문하겠냐 물어보니 침착맨 추천 맛집 칼국수도 만두도 정말 직접 손으로 만든다는 느낌이 강한 요즘은 드문 진짜 인생맛집 칼국수입니다 면도 직접 만드시고 만두도 직접 만드시고 김치도 직접 만드시고 맛 갑니다 양은 매우 많고 맛도 좋습니다 검색하고 별 기대안하고 간 집인데 칼국수 맛집이네요 칼국수의 조미료맛이 거의 안나고 국물이 시원합니다 되었네요 맛집이라 그런지 점심시간에 가서 석계 맛집 특히 면과 겉절이가 맛있 맛집인지 팀 대기후 자리에 앉았음 직접 반죽해서 맛나서 한번씩 생각나서 여기 진짜 찐맛집인생 칼국수집 석계갈 저는 별로였습니다 바지락 칼국수의 국물은 혼다시 맛이 강하고 닭칼국수의 맛은 스톡 맛이 강한게 로였습니다 바지락 칼국수의 국물은 혼다시 맛이 강하고 닭칼국수의 맛은 스톡 맛이 강한게 좀 아쉬운 부분이에요 만두는 잘 싸여져 있는 만두가 아니라 소에 둘러놓은 느낌이라 판에서 땔때 분리가 되더라구요 가장 나은거는 밀가루 입니다 국수 면은 약간 두꺼운 진짜 석계 맛집 친절하시고 깨끗하고 석계에 몇 없는 

In [ ]:
# 최종 데이터프레임 생성
result_df = pd.DataFrame(result)
result_df

,restaurant,category,summary,real
0,무명칼국수,맛,"칼국수면발이 아주 쫄깃해서 식감이 좋고, 석계 맛집 해물도 닭고 축복임 맛 서비스 ...",맛 전혀 특별하지 않음 해물칼국수 정말 괜찮음 닭칼은 그닥 만두는 부추맛이 강해서 ...
1,무명칼국수,가격,가게됨 가격도 착한편 맛과 가격 모두 괜찮은 집 너무 맛있는데 가격까지 저렴해요. ...,가게됨 가격도 착한편 맛과 가격 모두 괜찮은 집 너무 맛있는데 가격까지 저렴해요 가...
2,무명칼국수,서비스,겉절이김치가 넘 맛있고 사장님과 직원분도 친절하세요 있었는데 서비스로 주셨어용 맛도...,축복임 맛 서비스 모두 최고 직원분들도 친절해서 더욱 좋았네요 사장님이 너무 친절해...
3,무명칼국수,분위기,"정겨운 시골 식당 같아서 좋고, 주변 경치가 너무 칙칙해서 별로입니다. 칼국수 집에...",약간 시골 분위기라서 데이트하긴 별로에요 간단하게 점심먹기 딱 좋아요 친구들하고 밥...
4,연어시장 석계본점,맛,연어매운탕도 맛있고 기본전이랑 떡볶이도 좋아요 단 맛이 도는 그 기름기 머금은 연어...,떡볶이도 맛집이에요 다른 식사 메뉴를 먹을 수도 있었 연어맛집 연어매운탕도 맛있고 ...
5,연어시장 석계본점,서비스,맛나고 직원들도 친절해요 어디선가 식초냄새가 훅훅 심하게 나는데 맛있어요 진짜 서비...,맛나고 직원들도 친절해요 어디선가 식초냄새가 훅훅 심하게나는데 맛있어요 진짜 서비스...
6,연어시장 석계본점,가격,무한리필로 연어랑 참치를 이 가격에 먹을 수 있다는게 너무 좋아요. 가격도 저렴하고...,됨 아쉬운 건 사이드 총합 가격이 조금 서울에 이만한 연어 무한리필집 없다 심지어 ...
7,연어시장 석계본점,분위기,"술먹기 좋고 분위기도 좋은 석계에서 가장 좋은 분위기입니다. 조명이 잘나와서 좋고,...",분위기 너무 좋아요 술먹기 좋은 분위기에요 너무 좋은 분위기 석계에서 가장 좋은 분...
8,한상,분위기,석관동에 이런 뷰가 있네하면서 몰지각이 이루 말할수없음 뷰는 트였고 다만 한정식집 ...,분정도 뷰에대해 감탄했고 석관동에 이런 뷰가있네하면서 몰지각이 이루 말할수없음 앞접...
9,한상,가격,가격이 비싼데 나오는 반찬들이 가지수만 많고 반찬 구성에 대해 말이 많던데 이 가격...,엄마가놀래서 가방을 붙잡음 천만원짜리 가방임 가방가격이 중요한게 여기망할듯 음식금액...


In [ ]:
result_df.to_csv('/data/summary_category_update.csv', index=False)